# PZ DC1 Metrics Script
_Sam Schmidt, Alex Malz, Rongpu Zhou, Chris Morrison, Karthiek Iyer, Bryce Kalmbach, Jeff Newman_

The purpose of this script is to demonstrate the use of the metrics scripts to be used on the photo-$z$ PDF catalogs produced by the PZ working group for the DC1 paper.

To run this code, you must install [`qp`](https://github.com/aimalz/qp) and have the notebook in the same directory as `individual_metrics.py`.  You must also install some run-of-the-mill Python packages: `matplotlib`, `numpy`, `scipy`, and `skgof`.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import qp
import individual_metrics as inmet
import skgof
%matplotlib inline

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Warning: number of zref values not equal to number of ensemble objects')? (individual_metrics.py, line 45)

We'll start with two Gaussian distributions defined in some interval.
_What is the significance of the constant coefficients on the PDFs?  Why not just use normalized PDFs everywhere?  --aimalz_

In [ ]:
x = np.arange(0.5,10.0,0.02)
func1 = ss.norm(loc=3.0,scale=0.5)
func2= ss.norm(loc=5.0,scale=0.35)
# y1 = func1.pdf(x)*20000./0.25*0.1
# y2 = func2.pdf(x)*40000./0.35*0.1
y1 = 1.*func1.pdf(x)
y2 = 2.*func2.pdf(x)

We take samples from them and plot a histogram to confirm the samples follow the distribution.

In [ ]:
# n_samp1, n_samp2 = 20000, 40000
n_samp1, n_samp2 = 20, 40
# samples1 = np.array(func1.rvs(size=n_samp1))
# samples2 = np.array(func2.rvs(size=n_samp2))
# plt.hist(samples1, normed=False, histtype='stepfilled', alpha=0.2)
# plt.hist(samples2, normed=False, histtype='stepfilled', alpha=0.2)
samples1 = np.array(func1.rvs(size=n_samp1))
samples2 = np.array(func2.rvs(size=n_samp2))
plt.hist(samples1, normed=True, histtype='stepfilled', alpha=0.2)
plt.hist(samples2, normed=True, histtype='stepfilled', alpha=0.2)
plt.plot(x,y1,lw=2,c='r')
plt.plot(x,y2,lw=2,c='b')

When we combine all the samples together, we have samples from a two-component Gaussian mixture.  These will be the true redshifts $\{z_{i}\}_{N}$ of our catalog.

In [ ]:
bigsample = np.append(samples1,samples2)
n_tot = len(bigsample)
print(n_tot)
assert(n_tot == n_samp1 + n_samp2)

Now we populate a catalog of PDFs $\{p_{i}(z)\}_{N}$ using the components of the Gaussian mixture evaluated on a grid using the `qp.Ensemble` class.

In [ ]:
smallgrid = np.ones([n_tot,len(y1)])
# for i in range(n_samp1):
#     smallgrid[i,:] = y1
# for i in range(n_samp1, n_tot):
#     smallgrid[i,:] = y2
smallgrid[:n_samp1] *= y1[np.newaxis, :]
smallgrid[n_samp1:] *= y2[np.newaxis, :]
approx_pdf = qp.Ensemble(n_tot, gridded=(x, smallgrid))

We create an `individual_metrics.EvaluateMetric` object from the catalog and the true redshifts.

In [ ]:
testme = inmet.EvaluateMetric(approx_pdf, bigsample)

The first metric we calculate is the Probability Integral Transform (PIT), 
\begin{equation*}
\mathrm{PIT}(p_{i}(z);\ z_{i})\ =\ \int_{-\infty}^{z_{i}}\ p_{i}(z)\ dz,
\end{equation*}
for every galxy $i$ in the catalog.

In [ ]:
testPIT = testme.PIT()

Next, we calculate the Kolmogorov-Smirnov (KS) test statistic,
\begin{equation*}
\mathrm{KS}(\{p_{i}(z)\}_{N};\ \{z_{i}\}_{N})\ =\ \max_{PIT}\left[ \left| CDF(\{PIT(p_{i}(z);\ z_{i})\}_{N}) - CDF(\{z_{i}\}_{N}) \right| \right],
\end{equation*}
on the distribution of PIT values, which should be uniform if the PDFs are perfect.

In [ ]:
ks_stat, ks_pval = testme.KS(using='gridded',dx=0.0001)
print ks_stat
print ks_pval

Similarly, we calculate the Cramer-von Mises (CvM) test statistic,
\begin{equation*}
\mathrm{CvM}(\{p_{i}(z)\}_{N};\ \{z_{i}\}_{N})\ =\ \int_{-\infty}^{\infty}\ \left(CDF(\{PIT(p_{i}(z);\ z_{i})\}_{N})\ -\ CDF(\{z_{i}\}_{N})\right)^{2}\ \mathrm{d}CDF(\{z_{i}\}_{N}),
\end{equation*}
on the distribution of PIT values, which should be uniform if the PDFs are perfect.

In [ ]:
cvm_stat,cvm_pval = testme.CvM(using='gridded',dx=0.0001)
print cvm_stat
print cvm_pval

And the Anderson-Darling (AD) test statistic,
\begin{equation*}
\mathrm{AD}(\{p_{i}(z)\}_{N};\ \{z_{i}\}_{N})\ =\ \int_{-\infty}^{\infty}\frac{\left(CDF(\{PIT(p_{i}(z);\ z_{i})\}_{N})\ -\ CDF(\{z_{i}\}_{N})\right)^{2}}{CDF(\{z_{i}\}_{N})\ \left(1\ -\ CDF(\{z_{i}\}_{N})\right)}\ \mathrm{d}CDF(\{z_{i}\}_{N}),
\end{equation*}
on the distribution of PIT values, which should be uniform if the PDFs are perfect.  However, for this test, we cut the ends of the distribution, which represent catastrophic utliers.  
_Note: I do not think we should perform this cut unless we don't care about the catastrophic outlier rate; if we do care and still perform this cut, we are losing our sensitivity to measuring it  --aimalz._

In [ ]:
ad_stat,ad_pval = testme.AD(using='gridded',dx=0.0001, vmin=0.01,vmax=0.99)
print ad_stat
print ad_pval

We can try this with different cutoff values in $CDF$.  Let's try a less conservative cut first.

In [ ]:
ad_stat,ad_pval = testme.AD(using='gridded',dx=0.0001, vmin=0.0001,vmax=0.9999)
print ad_stat
print ad_pval

And this is what we get without any cut.

In [ ]:
ad_stat,ad_pval = testme.AD(using='gridded',dx=0.0001, vmin=0.000,vmax=1.0)
print ad_stat
print ad_pval

Next we can calculate the CDE loss described in Izbicki & Lee 2017 (arXiv:1704.08095)

$$ \int \int ((p(z \mid x) - \hat{p}(z \mid x))^{2} dz dP(x) $$

which extends L2 density estimation loss to conditional density estimation.  We can estimate this quantity (up to an unknown additive constant which depends on the true conditional densities) from data as

$$ \frac{1}{n} \sum_{i=1}^{n} \int \hat{p}^{2}(z \mid x_{i}) dz - \frac{2}{n} \sum_{i=1}^{n} \hat{p}(z_{i} \mid x_{i}) $$



In [ ]:
cde_loss = testme.cde_loss(np.linspace(0.5, 10.0, 1000))
print cde_loss

We can make a QQ plot as well.

In [ ]:
qqplot = testme.QQplot(using='gridded',dx=0.0001,Nquants=101)

# Test with 111k galaxies from the training sample of the DC1 data set from BPZ

In [ ]:
infile = "/Users/samschmidt/PZPAPER/mystuff/test_magscat_trainingfile_probs.out"
szfile = "/Users/samschmidt/PZPAPER/mystuff/train_justsz.out"
z_array = np.arange(0.0050,2.1100,0.0100) #this is the grid output by BPZ for my runs
z_trues = np.loadtxt(szfile,skiprows=1)
alldata = np.loadtxt(infile,skiprows=1)
ID = alldata[:,0]
pzs = alldata[:,1:]
ngals = len(ID)

In [ ]:
bpz_approx_pdf = qp.Ensemble(pzs.shape[0],gridded=(z_array,pzs))

In [ ]:
bpzobj = inmet.EvaluateMetric(bpz_approx_pdf,z_trues)

In [ ]:
bpzPIT = bpzobj.PIT()

In [ ]:
ks_stat,ks_pval = bpzobj.KS(using='gridded',dx=0.0001)
print ks_stat
print ks_pval

In [ ]:
cvm_stat,cvm_pval=bpzobj.CvM(using='gridded',dx=0.0001)
print cvm_stat
print cvm_pval

In [ ]:
ad_stat,ad_pval=bpzobj.AD(using='gridded',dx=0.0001,vmin=0.05,vmax=0.95)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=bpzobj.AD(using='gridded',dx=0.0001,vmin=0.1,vmax=0.9)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=bpzobj.AD(using='gridded',dx=0.0001,vmin=0.0,vmax=1.0)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=bpzobj.AD(using='gridded',dx=0.0001,vmin=0.8,vmax=0.95)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=bpzobj.AD(using='gridded',dx=0.0001,vmin=0.8,vmax=1.0)
print ad_stat
print ad_pval

As an aside, the high p-values look good for BPZ, it's the low PIT percentiles that are off.  But, including the values of 1.0 does take the AD statistic back to a very large number, so outlier trimming is necessary

In [ ]:
bpzobj.QQplot(using='gridded',dx=0.0001,Nquants=1001)

In [ ]:
fig = plt.figure(figsize=(15,15))
plt.hist(bpzPIT, normed=False, histtype='stepfilled', alpha=0.7,bins=np.arange(0.0,1.01,.01))
plt.xlim([0.0,1.0])
plt.xlabel("PIT value",size=18)
plt.ylabel("Number",size=18)

# Make i<25.3 gold cut

In [ ]:
newfile = "train_idszmag.out"
newdata = np.genfromtxt(newfile)
mags = newdata[:,2]

In [ ]:
magcut = (mags<25.3)
goldzs = z_trues[magcut]
goldpzs = pzs[magcut]
goldnumgals = len(cutzs)
print goldnumgals
print goldpzs.shape

In [ ]:
gold_pdf = qp.Ensemble(goldpzs.shape[0],gridded=(z_array,goldpzs))

In [ ]:
goldbpzobj = inmet.EvaluateMetric(gold_pdf,goldzs)

In [ ]:
goldPIT = goldbpzobj.PIT()

In [ ]:
ks_stat,ks_pval = goldbpzobj.KS(using='gridded',dx=0.0001)
print ks_stat
print ks_pval

In [ ]:
cvm_stat,cvm_pval=goldbpzobj.CvM(using='gridded',dx=0.0001)
print cvm_stat
print cvm_pval

In [ ]:
ad_stat,ad_pval=goldbpzobj.AD(using='gridded',dx=0.0001,vmin=0.05,vmax=0.95)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=goldbpzobj.AD(using='gridded',dx=0.0001,vmin=0.1,vmax=0.9)
print ad_stat
print ad_pval

In [ ]:
ad_stat,ad_pval=goldbpzobj.AD(using='gridded',dx=0.0001,vmin=0.8,vmax=0.95)
print ad_stat
print ad_pval

In [ ]:
goldbpzobj.QQplot(using='gridded',dx=0.0001,Nquants=1001)

In [ ]:
fig = plt.figure(figsize=(15,15))
plt.hist(goldPIT, normed=False, histtype='stepfilled', alpha=0.7,bins=np.arange(0.0,1.01,.01))
plt.xlim([0.0,1.0])
plt.xlabel("gold sample PIT value",size=18)
plt.ylabel("Number",size=18)